# Brands Sentiment 

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")
# from plotnine import *
import re
from matplotlib import pyplot as plt
import matplotlib

In [2]:
df = pd.read_csv('C:\\Users\\hs324\\OneDrive\\Desktop\\Class_Files\\06_2022Fall\\04_Practicum\\Quantilope_Core\\data\\sentiment_score_text.csv',index_col=[0] )
df.head()

,roberta_neg,roberta_neu,roberta_pos,text
0,0.029508,0.537855,0.432637,@ShaneDaleAZ Totally. The Nike uniforms since ...
1,0.071209,0.369753,0.559038,Hats off to Tom Sachs and the marketing team a...
2,0.002428,0.077772,0.919799,The look on @makiracook face! 😂 \n\nThanks @Ni...
3,0.041717,0.864787,0.093497,Check out my new pickup from Nike⁠ SNKRS: http...
4,0.001843,0.033002,0.965156,@jameslfreelance @Jumpman23 @Nike @nikestore O...


In [3]:
# Since the sentiments adds up to 1, only need to choose the highest as the sentiment of this text. 
df.rename(columns = {'roberta_neg':'neg','roberta_neu':'neu','roberta_pos':'pos' },inplace = True )
df['sentiment'] = df[['neg','neu','pos']].idxmax(axis=1)
df

,neg,neu,pos,text,sentiment
0,0.029508,0.537855,0.432637,@ShaneDaleAZ Totally. The Nike uniforms since ...,neu
1,0.071209,0.369753,0.559038,Hats off to Tom Sachs and the marketing team a...,pos
2,0.002428,0.077772,0.919799,The look on @makiracook face! 😂 \n\nThanks @Ni...,pos
3,0.041717,0.864787,0.093497,Check out my new pickup from Nike⁠ SNKRS: http...,neu
4,0.001843,0.033002,0.965156,@jameslfreelance @Jumpman23 @Nike @nikestore O...,pos
...,...,...,...,...,...
11643,0.283802,0.640393,0.075805,"Armpit musty, Reebok crusty https://t.co/KjJ3z...",neu
11644,0.018777,0.809956,0.171267,@DoubleOhNegatve I believe that was the Reebok...,neu
11645,0.253581,0.710993,0.035426,If you ever find yourself talking shit on will...,neu
11646,0.046945,0.874524,0.078531,"Think the 4,5 shoes gon be Reebok, Doc’s and s...",neu


In [4]:
brands = ['Nike','Adidas','New Balance','Converse','Reebok']

In [5]:
for brand in brands:
    print(brand + ":" + str(df[df['text'].str.contains(brand, case = False, na = False)].shape[0]))

Nike:8069
Adidas:2650
New Balance:54
Converse:948
Reebok:341


In [6]:
# create column with brand names 
for brand in brands:
    df.loc[lambda x : x.text.str.lower().str.contains(brand.lower()), 'brand'] = brand.capitalize()
df 

,neg,neu,pos,text,sentiment,brand
0,0.029508,0.537855,0.432637,@ShaneDaleAZ Totally. The Nike uniforms since ...,neu,Nike
1,0.071209,0.369753,0.559038,Hats off to Tom Sachs and the marketing team a...,pos,Nike
2,0.002428,0.077772,0.919799,The look on @makiracook face! 😂 \n\nThanks @Ni...,pos,Nike
3,0.041717,0.864787,0.093497,Check out my new pickup from Nike⁠ SNKRS: http...,neu,Nike
4,0.001843,0.033002,0.965156,@jameslfreelance @Jumpman23 @Nike @nikestore O...,pos,Nike
...,...,...,...,...,...,...
11643,0.283802,0.640393,0.075805,"Armpit musty, Reebok crusty https://t.co/KjJ3z...",neu,Reebok
11644,0.018777,0.809956,0.171267,@DoubleOhNegatve I believe that was the Reebok...,neu,Reebok
11645,0.253581,0.710993,0.035426,If you ever find yourself talking shit on will...,neu,Reebok
11646,0.046945,0.874524,0.078531,"Think the 4,5 shoes gon be Reebok, Doc’s and s...",neu,Reebok


In [22]:
# group by brand and sentiment 
brand_sentiment = df.groupby(['brand','sentiment'])['text'].count()
brand_sentiment = brand_sentiment.groupby(level=0).apply(lambda X: 100 * X / float(X.sum())).reset_index().rename(columns = {'text': 'percentage'})
brand_sentiment 

,brand,sentiment,percentage
0,Adidas,neg,13.008779
1,Adidas,neu,66.759777
2,Adidas,pos,20.231445
3,Converse,neg,18.326271
4,Converse,neu,59.216102
5,Converse,pos,22.457627
6,New balance,neg,10.869565
7,New balance,neu,65.217391
8,New balance,pos,23.913043
9,Nike,neg,13.774888


In [26]:
# most positive 
brand_sentiment[brand_sentiment['sentiment'] == 'pos'].sort_values(by = 'percentage', ascending = False)

,brand,sentiment,percentage
8,New balance,pos,23.913043
5,Converse,pos,22.457627
11,Nike,pos,20.926453
2,Adidas,pos,20.231445
14,Reebok,pos,10.850440


In [27]:
# most negative 
brand_sentiment[brand_sentiment['sentiment'] == 'neg'].sort_values(by = 'percentage', ascending = False)

,brand,sentiment,percentage
3,Converse,neg,18.326271
9,Nike,neg,13.774888
0,Adidas,neg,13.008779
6,New balance,neg,10.869565
12,Reebok,neg,5.278592
